In [181]:
import cv2
import numpy as np

filename = "building1.jpg"
win_name = "scanning"
img = cv2.imread(filename)
rows, cols = img.shape[:2]
draw = img.copy()
pts_cnt = 0
pts = np.zeros((6, 2), dtype=np.float32)

imgcropleft = img.copy()
imgcropright = img.copy()

def onMouse(event, x, y, flags, param):
    global pts_cnt
    if event == cv2.EVENT_LBUTTONDOWN:
        # 좌표에 초록색 동그라미 표시
        cv2.circle(draw, (x, y), 10, (0, 255, 0), -1)
        cv2.imshow(win_name, draw)

        # 마우스 좌표 저장
        pts[pts_cnt] = [int(x), int(y)]
        pts_cnt += 1
        if pts_cnt == 6:
            # 좌표 6개 위치 찾기
               
            #중심점 x좌표
            centerx = pts[2][0]
            
            #centerx를 기준으로 건물의 면을 분리
            imgcropleft = img[0:,0:round(centerx)]
            imgcropright = img[0:,round(centerx):]

            
            #movement, 픽셀 변화율
            t = 10

            #temp, 좌우 변형 후 보간 변수
            temp = 10
            
            while True:
                
                #좌측면 변형 전후 이미지 점
                leftpts1 = np.float32([pts[0], pts[2], pts[1], pts[3]])
                leftpts2 = np.float32([pts[0], [pts[2][0] + t, pts[2][1]], pts[1], [pts[3][0] + t, pts[3][1]]])
        
                #우측면 변형 전후 이미지 점
                rightpts1 = np.float32([pts[0], pts[2], pts[1], pts[3]])
                rightpts2 = np.float32([pts[0], [pts[2][0] - t, pts[2][1]], pts[1], [pts[3][0] - t, pts[3][1]]])
                
                mtrx1 = cv2.getPerspectiveTransform(leftpts1, leftpts2)
                mtrx2 = cv2.getPerspectiveTransform(rightpts1, rightpts2)

                
                result1 = cv2.warpPerspective(imgcropleft, mtrx1, (int(pts[2][0]) + t, 1200))
                result2 = cv2.warpPerspective(imgcropright, mtrx2, (int(pts[2][0]) - t, 1200))
                result_1 = np.hstack([result1, result2[:,temp:,:]])
                cv2.imshow('1', result_1)

                #키보드 입력 감지
                key = cv2.waitKey()

                if key == ord('='):
                    if temp - 1 >= -1 * centerx and temp - 1 >= 0:
                        temp = temp - 1
                elif key == ord('-'):
                    temp = temp + 1
                elif key == ord('['):
                    t = t - 1
                elif key == ord(']'):
                    t = t + 1
                elif key == 27:
                    cv2.destroyAllWindows()
                    break
            
cv2.imshow(win_name, img)
cv2.setMouseCallback(win_name, onMouse)
cv2.waitKey(0)
cv2.destroyAllWindows()